In [1]:
import pandas as pd
import numpy as np
import pickle
root="data/"

In [2]:
train_sample = pickle.load(open(root+"nlfl_train_sample_v3.pickle", "rb"))
val_sample = pickle.load(open(root+"nlfl_val_sample_v3.pickle", "rb"))
test_sample = pickle.load(open(root+"nlfl_test_sample_v3.pickle", "rb"))

In [3]:
file_names = {
    "mf_train": root+"mf_features_train_task_C1.xlsx",
    "mf_test": root+"mf_features_test_task_C1.xlsx",
    "mf_val": root+"mf_features_val_task_C1.xlsx",
}

In [4]:
%%time
mf_A_train = pd.read_excel(file_names["mf_train"])
mf_A_val = pd.read_excel(file_names["mf_val"])
mf_A_test = pd.read_excel(file_names["mf_test"])

CPU times: user 1min 54s, sys: 153 ms, total: 1min 54s
Wall time: 1min 54s


In [5]:
%%time
mf_A_train = mf_A_train.set_index("id").loc[train_sample["id"]].reset_index()
mf_A_test = mf_A_test.set_index("id").loc[test_sample["id"]].reset_index()
mf_A_val = mf_A_val.set_index("id").loc[val_sample["id"]].reset_index()

CPU times: user 53.1 ms, sys: 48 ms, total: 101 ms
Wall time: 104 ms


In [6]:
cols_mf = [c for c in mf_A_train.columns if "linguistic" not in c and c != "id"]
cols_nlfl = [c for c in train_sample.columns if "chatgpt_" in c or c in ["id", "label_Q", "label"]]
cols = cols_mf + cols_nlfl

In [7]:
%%time
nlfl_hf_train = pd.concat([train_sample[cols_nlfl], mf_A_train[cols_mf]], axis=1)
nlfl_hf_test = pd.concat([test_sample[cols_nlfl], mf_A_test[cols_mf]], axis=1)
nlfl_hf_val = pd.concat([val_sample[cols_nlfl], mf_A_val[cols_mf]], axis=1)

CPU times: user 41.2 ms, sys: 16 ms, total: 57.2 ms
Wall time: 56.6 ms


In [56]:
k = 0
for i in train_sample[train_sample["label"] == 0].sample(10).index:
    k += 1
    print(f"Ejemplo {k}:\n")
    print("Pregunta:", train_sample.loc[i]["Q"])
    print("Respuesta:", train_sample.loc[i]["A"])
    print("\n")

Ejemplo 1:

Pregunta:  Daniel quiere llenar su piscina con agua potable. La piscina tiene las siguientes dimensiones: 6 metros de ancho, 3 metros de largo y 4 metros de alto. Daniel dice que necesitará 13 metros cúbicos de agua. Explica por qué Daniel está equivocado.
Respuesta: EL NO NECESITA 13 NECESITA 72   


Ejemplo 2:

Pregunta: Si Mariela tiene 30 flores y su amiga Juanita le quita 10 flores, ¿cuántas flores tiene ahora Mariela?
Respuesta: mariela ahora tiene 20



Ejemplo 3:

Pregunta: Andrea tiene 200 pesos. Su abuela le regala un sobre con una cantidad de dinero desconocida. Ahora Andrea tiene en total $550. ¿Cuánto dinero le dio la abuela a Andrea? Explica con tus palabras cómo llegaste al resultado y además escribe la ecuación correspondiente. 
Respuesta: la ecuación es
200+z=550
y yo le quite y me dio 
550-200=350


Ejemplo 4:

Pregunta: Juan y Diego compran cada uno una torta del mismo tamaño. Juan se come ½ de su torta y Diego 2/4 de la de él. ¿Cuál de los dos niños comi

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
tfidf = TfidfVectorizer(ngram_range=(1,2), max_features=1000, min_df=1, stop_words=[])

In [10]:
X_bong_train = np.asarray(tfidf.fit_transform(train_sample["A"].apply(str)).todense())
X_bong_test = np.asarray(tfidf.transform(test_sample["A"].apply(str)).todense())
X_bong_val = np.asarray(tfidf.transform(val_sample["A"].apply(str)).todense())

In [1]:
hf_bong_nllf_train = pd.concat((nlfl_hf_train, pd.DataFrame(X_bong_train, columns=tfidf.get_feature_names())), axis=1)
hf_bong_nllf_test = pd.concat((nlfl_hf_test, pd.DataFrame(X_bong_test, columns=tfidf.get_feature_names())), axis=1)
hf_bong_nllf_val = pd.concat((nlfl_hf_val, pd.DataFrame(X_bong_val, columns=tfidf.get_feature_names())), axis=1)

NameError: name 'pd' is not defined

In [2]:
from sklearn.tree import DecisionTreeClassifier

cols_nlfl = [c for c in hf_bong_nllf_train.columns if "chatgpt_" in c or c in ["id", "label_Q", "label"]]
cols = cols_nlfl + tfidf.get_feature_names()+cols_mf

new_cols = [c for c in cols if c not in ["id", "label", "label_Q", "chatgpt_label"]] 
new_cols

NameError: name 'hf_bong_nllf_train' is not defined

In [14]:
new_best_features = ['chatgpt_v1 (Y)',
 'chatgpt_v2 (Y)',
 'chatgpt_v4 (Y)',
 'chatgpt_v5 (Y)',
 'chatgpt_v6 (Y)',
 'chatgpt_v7 (N)',
 'chatgpt_v8 (N)',
 'chatgpt_v9 (N)',
 'chatgpt_v9 (Y)',
 'chatgpt_v10 (N)',
 'chatgpt_v5_1 (N)',
 'chatgpt_v5_1 (Y)',
 'chatgpt_v5_2 (N)',
 'chatgpt_v5_3 (N)',
 'chatgpt_v5_4 (N)',
 'chatgpt_v6_2 (N)',
 'chatgpt_v6_4 (N)',
 'chatgpt_Q3_1 (N)',
 'chatgpt_Q3_1 (Y)',
 'chatgpt_Q3_4 (N)',
 'chatgpt_Q3_4 (Y)',
 'chatgpt_Q3_5 (N)',
 'chatgpt_Q3_6 (N)',
 'chatgpt_Q3_7 (N)',
 'chatgpt_Q3_8 (Y)',
 'chatgpt_Q3_9 (N)',
 'chatgpt_Q3_9 (Y)',
 'chatgpt_Q3_10 (Y)',
 'chatgpt_Q3_11 (N)',
 'chatgpt_Q3_13 (Y)',
 'chatgpt_Q3_14 (Y)',
 'chatgpt_Q3_16 (N)',
 'chatgpt_Q3_16 (Y)',
 'chatgpt_Q3_7_e1 (N)',
 'chatgpt_Q3_7_e1 (Y)',
 'chatgpt_Q3_7_e2 (N)',
 'chatgpt_Q3_7_e2 (Y)',
 'chatgpt_Q3_7_e3 (N)',
 'chatgpt_Q3_7_e4 (N)',
 'chatgpt_Q3_7_e4 (Y)',
 'chatgpt_Q3_7_e5 (N)',
 'chatgpt_10_e1 (Y)',
 'chatgpt_10_e2 (N)',
 'chatgpt_10_e2 (Y)',
 'chatgpt_10_e4 (Y)',
 'chatgpt_10_e5 (Y)',
 'chatgpt_7_e2 (N)',
 'chatgpt_7_e2 (Y)',
 'chatgpt_7_e3 (N)',
 'chatgpt_7_e3 (Y)',
 'chatgpt_7_e4 (Y)',
 'chatgpt_7_e5 (Y)',
 'chatgpt_Q3_4_e1 (Y)',
 'chatgpt_Q3_4_e2 (Y)',
 'chatgpt_Q3_4_e3 (Y)',
 'chatgpt_Q3_4_e4 (N)',
 'chatgpt_Q3_4_e4 (Y)',
 'chatgpt_Q3_4_e5 (N)',
 'chatgpt_Q3_4_e5 (Y)',
 'chatgpt_Q3_5_e1 (N)',
 'chatgpt_Q3_5_e2 (N)',
 'chatgpt_Q3_5_e3 (N)',
 'chatgpt_Q3_5_e5 (N)',
 'chatgpt_HF_3 (N)',
 'chatgpt_HF_3 (Y)',
 'chatgpt_HF_6 (Y)',
 'chatgpt_SP_1 (Y)',
 'chatgpt_SP_3 (N)',
 'chatgpt_SP_4 (N)',
 'chatgpt_SP_4 (Y)',
 'chatgpt_SP_5 (Y)',
 'chatgpt_SP_6 (N)'] + ['contextual<&>Q[quién|cuál|qué]',
 'contextual<&>Q[ser&posible]',
 'contextual<&>A[binary(si|no)]',
 'contextual<&>Q[por&qué&equivocado]',
 'contextual<&>Q[ser*&bien]',
 'contextual<&>Q[propn]+&A+',
 'contextual<&>Q[quién|cuál]',
 'contextual<&>Q[ser*&correcto]',
 'contextual<&>Q[tener&razón]',
 'traditional<&>prop_punct+no-vowel',
 'traditional<&>prop_vowels',
 'traditional<&>len_max_rep_char',
 'traditional<&>frec_char(g)',
 'traditional<&>frec_char(h)',
 'traditional<&>frec_char(x)',
 'traditional<&>frec_char(w)',
 'traditional<&>frec_char(ñ)',
 'semantic<&>A.is(ola|hola)',
 'semantic<&>A.contains(bad-word)',
 'semantic<&>prop_keywords',
 'traditional<&>prop_digit_char',
 'traditional<&>num_tokens',
 'traditional<&>num_digit',
 'semantic<&>num_rae',
 'semantic<&>num_ud',
 'semantic<&>num_slang',
 'semantic<&>num_faces',
 'traditional<&>ratio_vowel',
 'traditional<&>exist_numbs']+ ['00',
 '10 de',
 '10 es',
 '10 le',
 '10 no',
 '10 porque',
 '100 chocolates',
 '12 12',
 '12 es',
 '12 sacos',
 '15',
 '150',
 '1590',
 '16',
 '17',
 '20',
 '20 00',
 '200',
 '200 550',
 '22',
 '25',
 '2900',
 '35',
 '35 minutos',
 '350 pesos',
 '37',
 '40',
 '401',
 '45',
 '50',
 '500 000',
 '500 30',
 '500 500',
 '5200',
 '530',
 '550 200',
 '550 550',
 '56',
 '60',
 '60 60',
 '600',
 '65',
 '650',
 '70',
 '700',
 '72',
 '75',
 '750',
 '80',
 '90 grados',
 '932',
 '980',
 'abuela',
 'abuela le',
 'aceite',
 'agua',
 'agudo',
 'ahora tiene',
 'ai',
 'al lado',
 'al norte',
 'al resultado',
 'ala',
 'alambre',
 'alto',
 'amiga',
 'amigos',
 'ana',
 'angulo',
 'area',
 'autitos',
 'ay',
 'azucar',
 'balanza',
 'balones',
 'banderas',
 'bien porque',
 'bolitas',
 'cada',
 'cada dia',
 'cada niño',
 'cajas',
 'camila es',
 'camila por',
 'camila tiene',
 'carla',
 'carmen',
 'casa',
 'catalina ocupo',
 'cebollas',
 'cecilia',
 'cemento',
 'cesar',
 'cesilia',
 'chirimoya',
 'cinco',
 'claudio',
 'cm',
 'coma',
 'come',
 'comer',
 'comida',
 'comieron lo',
 'comio',
 'comió',
 'compre',
 'compro',
 'con 10',
 'con 500',
 'con la',
 'correcta',
 'creo',
 'creo que',
 'cuaderno',
 'cuadrado',
 'cuanto',
 'cuantos',
 'cuantos huevos',
 'cuatro',
 'cuerpo',
 'dar',
 'de',
 'de 90',
 'de agua',
 'de alambre',
 'de carlos',
 'de la',
 'de los',
 'de torta',
 'debe',
 'decimas',
 'decimos',
 'demas',
 'demora',
 'derecha',
 'despues',
 'di cuenta',
 'dia',
 'dibuje',
 'dice',
 'diego',
 'dio 10',
 'dio 350',
 'dise',
 'dividir',
 'don',
 'dos',
 'dos comieron',
 'dulces',
 'ecuacion',
 'eduardo',
 'ejemplo',
 'el aceite',
 'el alto',
 'el de',
 'el denominador',
 'el largo',
 'el mismo',
 'el numero',
 'empanadas',
 'en cada',
 'en el',
 'en la',
 'en lo',
 'en octubre',
 'en partes',
 'en total',
 'en un',
 'en una',
 'encuesta',
 'encuesta es',
 'entotal',
 'equivocada',
 'equivocada por',
 'equivocado',
 'equivoco',
 'era',
 'eran',
 'es 15',
 'es 20',
 'es 35',
 'es 350',
 'es 45',
 'es como',
 'es correcto',
 'es el',
 'es igual',
 'es mas',
 'es mayor',
 'es menor',
 'es ta',
 'es un',
 'es una',
 'esa',
 'ese resultado',
 'eso es',
 'esta equibocada',
 'esta equivocado',
 'esta formado',
 'este',
 'faltan',
 'figura',
 'flores en',
 'formado',
 'francisca',
 'gabriel',
 'gana',
 'grados',
 'grafico',
 'grande',
 'grandes',
 'hacia',
 'hay que',
 'igual que',
 'incognita',
 'inecuacion',
 'inversa',
 'jaime',
 'jose',
 'juan comio',
 'juan porque',
 'juan se',
 'juan tiene',
 'juanita',
 'kilos',
 'la abuela',
 'la cantidad',
 'la coma',
 'la de',
 'la granja',
 'la mitad',
 'la operacion',
 'la otra',
 'la pizza',
 'la rason',
 'la respuesta',
 'la resta',
 'la torta',
 'lados',
 'las',
 'las fracciones',
 'le',
 'le dio',
 'le falta',
 'le faltan',
 'le quedaron',
 'le quedo',
 'le sobro',
 'lentejas',
 'limones',
 'llegar',
 'lo',
 'lo corecto',
 'lo hice',
 'lo ise',
 'lo mismo',
 'lo multiplique',
 'lo supe',
 'lorena',
 'los dos',
 'los numeros',
 'los números',
 'luego',
 'mal',
 'mal porque',
 'mama esta',
 'manzanas',
 'marcia',
 'maria',
 'mariela tiene',
 'marta',
 'martin',
 'mas',
 'mas de',
 'mas es',
 'mas grandes',
 'mas jugo',
 'mas porque',
 'matematicas',
 'me',
 'me da',
 'me dio',
 'me salio',
 'medir',
 'menor',
 'menor que',
 'menos',
 'menos que',
 'metro',
 'mil',
 'minutos en',
 'misma cantidad',
 'multiplica',
 'multiplicar',
 'multiplico',
 'multiplique',
 'multiplique por',
 'ninguna',
 'no',
 'no es',
 'no por',
 'no porque',
 'no puede',
 'no tiene',
 'nos',
 'nota',
 'numeros',
 'números',
 'octubre',
 'ocupo cebollas',
 'otros',
 'pablo le',
 'paola',
 'para aser',
 'para llegar',
 'partes',
 'partes iguales',
 'pasos al',
 'patricia',
 'paz',
 'pedaso',
 'pedasos',
 'pedasos de',
 'pepo tiene',
 'perimetro',
 'pero',
 'personas',
 'pesos',
 'pierde',
 'poco',
 'podemos',
 'pondra',
 'poner',
 'por',
 'por el',
 'por es',
 'por eso',
 'por la',
 'por me',
 'por que',
 'porce',
 'porque 10',
 'porque el',
 'porque es',
 'porque esta',
 'porque la',
 'porque las',
 'porque le',
 'porque lo',
 'porque los',
 'porque para',
 'porque se',
 'porque si',
 'pregunta',
 'primero',
 'profesor',
 'profesor de',
 'puede hacer',
 'puede ser',
 'pueden',
 'pusieron',
 'puso',
 'que',
 'que 13',
 'que 24',
 'que ella',
 'que en',
 'que hay',
 'que le',
 'que lo',
 'que los',
 'que martina',
 'que multiplicar',
 'que no',
 'que por',
 'que restar',
 'que se',
 'que ser',
 'que si',
 'que son',
 'que sumar',
 'que tenia',
 'que tiene',
 'que yo',
 'queda',
 'quedan',
 'quedan 10',
 'quedaron 10',
 'rason',
 'razon',
 'razon porque',
 'razón',
 'recto',
 'regalo',
 'repartio',
 'respuesta es',
 'resta',
 'restamos',
 'reste 10',
 'resto',
 'resultado',
 'rotacion',
 'sabe',
 'saber',
 'sale',
 'salga',
 'se comieron',
 'se demora',
 'se demoro',
 'se le',
 'se multiplica',
 'se puede',
 'serca',
 'seria',
 'si esta',
 'si le',
 'si lo',
 'si porque',
 'sin',
 'sin comer',
 'sofia',
 'sofia esta',
 'solo',
 'son',
 'son 10',
 'su',
 'su abuela',
 'suma',
 'sumar',
 'sumo',
 'supe',
 'sur',
 'sus',
 'sus amigos',
 'ta',
 'tanto',
 'te',
 'te da',
 'tendria',
 'tenemos',
 'teresa tiene',
 'tiene',
 'tiene 20',
 'tiene el',
 'tiene en',
 'tiene mas',
 'tiene rason',
 'tiene razon',
 'tienen',
 'tienes',
 'tienes que',
 'todos',
 'todos los',
 'tomo',
 'total',
 'total es',
 'total tiene',
 'tres octavos',
 'troso',
 'trozo',
 'trozos',
 'un angulo',
 'un trozo',
 'una',
 'una encuesta',
 'una fraccion',
 'una inecuacion',
 'una rotacion',
 'una suma',
 'uno',
 'uso',
 'va',
 'vasos',
 'vendio 100',
 'volumen',
 'volumen de',
 'ya',
 'yo reste',
 'yo sume']

In [22]:
clf = DecisionTreeClassifier(random_state=42, min_impurity_decrease=0.0000, max_depth=5, criterion="gini")
clf.fit(hf_bong_nllf_train[new_best_features], hf_bong_nllf_train["label"])

DecisionTreeClassifier(max_depth=5, random_state=42)

In [23]:
from matplotlib import pyplot as plt
from sklearn import tree

In [24]:
fig, ax = plt.subplots(1, 1, figsize=(265, 85))
tree.plot_tree(clf, feature_names=new_best_features, ax=ax, fontsize=45, class_names=["C0","C1"]);

In [25]:
from sklearn.metrics import classification_report

In [26]:
print(classification_report(hf_bong_nllf_train["label"], clf.predict(hf_bong_nllf_train[new_best_features]), digits=4))

              precision    recall  f1-score   support

         0.0     0.9532    0.9770    0.9650     10019
         1.0     0.8216    0.6877    0.7487      1540

    accuracy                         0.9385     11559
   macro avg     0.8874    0.8324    0.8568     11559
weighted avg     0.9356    0.9385    0.9361     11559



In [27]:
val_sub_index = hf_bong_nllf_val.index
print(classification_report(hf_bong_nllf_val.loc[val_sub_index]["label"], clf.predict(hf_bong_nllf_val.loc[val_sub_index][new_best_features]), digits=4))

              precision    recall  f1-score   support

           0     0.9497    0.9769    0.9631      2510
           1     0.8165    0.6649    0.7330       388

    accuracy                         0.9351      2898
   macro avg     0.8831    0.8209    0.8480      2898
weighted avg     0.9318    0.9351    0.9323      2898



In [28]:
sub_index_test = hf_bong_nllf_test.index#[bong_nllf_test["label_Q"] == 3].index
print(classification_report(hf_bong_nllf_test.loc[sub_index_test]["label"], clf.predict(hf_bong_nllf_test.loc[sub_index_test][new_best_features]), digits=4))

              precision    recall  f1-score   support

           0     0.9392    0.9704    0.9545       541
           1     0.8644    0.7500    0.8031       136

    accuracy                         0.9261       677
   macro avg     0.9018    0.8602    0.8788       677
weighted avg     0.9242    0.9261    0.9241       677

